$$
M_a = \frac{1}{n} \sum_{i=1}^{n} x_i
$$

In [ ]:
from binance.spot import Spot
import time
import pandas as pd
import numpy as np

client = Spot(base_url='https://testnet.binance.vision', api_key='OMISSIS', api_secret='OMISSIS')

df = pd.DataFrame()
count1 = 0
count2 = 0
df['op'] = 0
df['count'] = 0
df['exe'] = 0

buy = {
    'symbol': 'DOGEUSDT',
    'side': 'BUY',
    'type': 'MARKET',
    'quantity': 100.000
}

sell = {
    'symbol': 'DOGEUSDT',
    'side': 'SELL',
    'type': 'MARKET',
    'quantity': 100.000
}

print(client.account())

def dati():
    global df
    price = client.avg_price(symbol = 'DOGEUSDT')
    price_df = pd.json_normalize(price)
    df = pd.concat([df, price_df])
    df.reset_index(inplace=True, drop=True)
    df['op'] = df['op'].fillna(0)

def media_mobile():
    df["x"]=df["price"].rolling(window=5, min_periods=2).mean()
    df["y"]=df["price"].rolling(window=21, min_periods=4).mean()
    df['x'] = df['x'].fillna(0)
    df['y'] = df['y'].fillna(0)

def logica():
    global count1
    global count2
    count1 = count1 + 1
    # Se x > y e x < y a t-1 compra
    if df['x'].loc[x] > df['y'].loc[x]:
        if x == 0:
            y = x
        if x > 0:
            y = x - 1
            if df['x'].loc[y] <= df['y'].loc[y]:
                client.new_order(**buy)
                df['count'].loc[x] = count1
    # Se x > y e x > y a t-1 neutro
    if df['x'].loc[x] > df['y'].loc[x]:
        if x == 0:
            y = x
        if x > 0:
            y = x - 1
            if df['x'].loc[y] > df['y'].loc[y]:
                df['op'].loc[x] = 0
    # Se x < y e x > y a t-1 vende
    if df['x'].loc[x] <= df['y'].loc[x]:  
        if x == 0:
            y = x
        if x > 0:
            y = x - 1
            if df['x'].loc[y] > df['y'].loc[y]:
                client.new_order(**sell)
                df['count'].loc[x] = count1
    # Se x < y e x < y a t-1 neutro
    if df['x'].loc[x] <= df['y'].loc[x]:
        if x == 0:
            y = x
        if x > 0:
            y = x - 1
            if df['x'].loc[y] <= df['y'].loc[y]:
                df['op'].loc[x] = 0
             
'''
def esecuzione():
    global count2
    count2 = count2 + 1
    if df['op'].loc[x] == 1:
        df['exe'] = 1
    if df['op'].loc[x] == -1:
        df['exe'] = -1
'''

for x in range(0, 60):
    time.sleep(60)
    dati()
    media_mobile()
    logica()
    
print(client.account())